In [136]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

np.random.seed(123)

In [137]:
# NETWORK = 'InceptionResNetV2'
# NETWORK = 'InceptionV3'
# NETWORK = 'Resnet'
# NETWORK = 'VGG16'
# NETWORK = 'VGG19'
# NETWORK = 'Xception'
# NETWORK = 'DenseNet201'
# NETWORK = 'EfficientNetB1'
# NETWORK = 'EfficientNetB4'
NETWORK = 'EfficientNetB7'
# NETWORK = 'NASNetLarge'
# NETWORK = 'ConvNeXtBase'

METHOD = "STD-AE"

In [138]:
WORKING_DIRECTORY = '/datassd/WHOLEIMAGE_MAMIP'
PET_features_filename = f"Features_{NETWORK}_MA-MIP_WHOLE-IMAGE.csv"

features_path = os.path.join(WORKING_DIRECTORY, f"EXTRACTED_FEATURES/ExtractedFeatures_{METHOD}")
processed_features_path = os.path.join(WORKING_DIRECTORY, f"PROCESSED_FEATURES/ProcessedFeatures_{METHOD}")
if not os.path.exists(processed_features_path):
    os.makedirs(processed_features_path)


processed_features_filename = f"Processed_Features_{NETWORK}_MA-MIP_WHOLE-IMAGE.csv"
processed_features_full_filename = os.path.join(processed_features_path, processed_features_filename)

PET_featurs_full_path = os.path.join(features_path, PET_features_filename)

PET_filename = PET_features_filename.split(".")[0]

PET_outcome_data = pd.read_csv(PET_featurs_full_path)

outcome_file = "/datassd/WHOLEIMAGE_MAMIP/hecktor2022_patient_endpoint_training.csv"

In [139]:
PET_outcome_data.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,2550,2551,2552,2553,2554,2555,2556,2557,2558,2559
0,CHUM-001,-6.740956,2.538421,3.062779,-0.117218,-5.796555,-2.876704,4.290518,2.652739,8.520704,...,1.929274,-3.533120,4.870999,2.534870,3.075483,4.354334,-0.988191,4.636808,-0.461575,2.886159
1,CHUM-002,-6.883021,2.107330,4.072270,0.460693,-5.463542,-3.166687,5.390929,2.671386,9.435923,...,3.004794,-3.665313,6.701253,3.049496,3.974517,3.258802,-0.904965,4.981396,-0.900667,3.102600
2,CHUM-006,-7.966107,3.344303,4.422788,0.166495,-5.138462,-3.377617,4.389914,3.272475,7.764829,...,2.922431,-2.812337,4.988035,3.197684,3.953045,4.960732,-1.630694,4.045227,-0.364611,3.728268
3,CHUM-007,-6.794433,3.489818,4.198925,0.127618,-5.452459,-2.445157,4.088459,2.953079,8.057346,...,2.162660,-2.809754,5.047943,2.875894,3.037149,4.401493,-2.218180,4.051433,-0.589891,2.587678
4,CHUM-008,-7.624650,3.253284,4.747948,0.635085,-4.884399,-2.953063,4.520241,2.880921,8.334317,...,2.803300,-2.647574,6.058694,2.380784,4.074299,4.011816,-1.437855,4.254267,-0.057254,2.946161


# Up to here

Read Outcome csv

In [140]:
OD = pd.read_csv(outcome_file)
OD = OD.iloc[:,:]
OD.head()

,PatientID,Relapse,RFS
0,CHUM-001,0,1704
1,CHUM-002,1,439
2,CHUM-006,0,1186
3,CHUM-007,0,1702
4,CHUM-008,0,1499


In [141]:
OD = OD.rename(columns={'PatientID': 'Patient_ID'})
OD.head()

,Patient_ID,Relapse,RFS
0,CHUM-001,0,1704
1,CHUM-002,1,439
2,CHUM-006,0,1186
3,CHUM-007,0,1702
4,CHUM-008,0,1499


In [142]:
OD.columns[0]

'Patient_ID'

In [143]:
first_column = PET_outcome_data.columns[0]
PET_outcome_data = PET_outcome_data.rename(columns={first_column: 'Patient_ID'})

In [144]:
OD.columns[0]


'Patient_ID'

In [145]:
PET_outcome_data.columns[0]

'Patient_ID'

In [146]:
PET_outcome_data.head()

,Patient_ID,0,1,2,3,4,5,6,7,8,...,2550,2551,2552,2553,2554,2555,2556,2557,2558,2559
0,CHUM-001,-6.740956,2.538421,3.062779,-0.117218,-5.796555,-2.876704,4.290518,2.652739,8.520704,...,1.929274,-3.533120,4.870999,2.534870,3.075483,4.354334,-0.988191,4.636808,-0.461575,2.886159
1,CHUM-002,-6.883021,2.107330,4.072270,0.460693,-5.463542,-3.166687,5.390929,2.671386,9.435923,...,3.004794,-3.665313,6.701253,3.049496,3.974517,3.258802,-0.904965,4.981396,-0.900667,3.102600
2,CHUM-006,-7.966107,3.344303,4.422788,0.166495,-5.138462,-3.377617,4.389914,3.272475,7.764829,...,2.922431,-2.812337,4.988035,3.197684,3.953045,4.960732,-1.630694,4.045227,-0.364611,3.728268
3,CHUM-007,-6.794433,3.489818,4.198925,0.127618,-5.452459,-2.445157,4.088459,2.953079,8.057346,...,2.162660,-2.809754,5.047943,2.875894,3.037149,4.401493,-2.218180,4.051433,-0.589891,2.587678
4,CHUM-008,-7.624650,3.253284,4.747948,0.635085,-4.884399,-2.953063,4.520241,2.880921,8.334317,...,2.803300,-2.647574,6.058694,2.380784,4.074299,4.011816,-1.437855,4.254267,-0.057254,2.946161


In [147]:
# ranges_list = list()
# for col in PET_outcome_data.iloc[:, 1:]:
#     col_range = PET_outcome_data[col].max() - PET_outcome_data[col].min()
#     ranges_list.append(col_range)
#     print(f"Range of column {col}: {col_range}")
# print(f"maximum range: {max(ranges_list)}")

In [148]:
full_features_df = pd.merge(PET_outcome_data, OD, on=OD.columns[0])

In [149]:
full_features_df.head()

,Patient_ID,0,1,2,3,4,5,6,7,8,...,2552,2553,2554,2555,2556,2557,2558,2559,Relapse,RFS
0,CHUM-001,-6.740956,2.538421,3.062779,-0.117218,-5.796555,-2.876704,4.290518,2.652739,8.520704,...,4.870999,2.534870,3.075483,4.354334,-0.988191,4.636808,-0.461575,2.886159,0,1704
1,CHUM-002,-6.883021,2.107330,4.072270,0.460693,-5.463542,-3.166687,5.390929,2.671386,9.435923,...,6.701253,3.049496,3.974517,3.258802,-0.904965,4.981396,-0.900667,3.102600,1,439
2,CHUM-006,-7.966107,3.344303,4.422788,0.166495,-5.138462,-3.377617,4.389914,3.272475,7.764829,...,4.988035,3.197684,3.953045,4.960732,-1.630694,4.045227,-0.364611,3.728268,0,1186
3,CHUM-007,-6.794433,3.489818,4.198925,0.127618,-5.452459,-2.445157,4.088459,2.953079,8.057346,...,5.047943,2.875894,3.037149,4.401493,-2.218180,4.051433,-0.589891,2.587678,0,1702
4,CHUM-008,-7.624650,3.253284,4.747948,0.635085,-4.884399,-2.953063,4.520241,2.880921,8.334317,...,6.058694,2.380784,4.074299,4.011816,-1.437855,4.254267,-0.057254,2.946161,0,1499


In [150]:
full_features_df.to_csv(processed_features_full_filename, encoding='utf-8', index=False)